In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"


In [0]:
%run "../includes/common_functions"


In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, TimestampType, FloatType

results_schema = StructType(fields = [
    StructField("resultId", IntegerType(), False),
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("constructorId", IntegerType(), True),
    StructField("number", IntegerType(), True),
    StructField("grid", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("positionText", StringType(), True),
    StructField("positionOrder", IntegerType(), True),
    StructField("points", FloatType(), True),
    StructField("laps", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True),
    StructField("fastestLap", IntegerType(), True),
    StructField("rank", IntegerType(), True),
    StructField("fastestLapTime", StringType(), True),
    StructField("fastestLapSpeed", FloatType(), True),
    StructField("statusId", StringType(), True)
])

In [0]:
results_df = spark.read.schema(results_schema).json(f"{raw_folder_path}/{v_file_date}/results.json")
display(results_df)

In [0]:
from pyspark.sql.functions import col, concat, lit

results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id") \
                                    .withColumnRenamed("raceId", "race_id") \
                                    .withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("constructorId", "constructor_id") \
                                    .withColumnRenamed("positionText", "position_text") \
                                    .withColumnRenamed("positionOrder", "position_order") \
                                    .withColumnRenamed("fastestLap", "fastest_lap") \
                                    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                    .withColumn("file_date", lit(v_file_date)) \
                                    .drop("statusId")
results_with_columns_df = add_ingestion_date(results_with_columns_df)
display(results_with_columns_df)

Method 1

In [0]:
# for race_id_list in results_with_columns_df.select("race_id").distinct().collect():
#     if spark.catalog.tableExists("f1_processed.results"):
#         spark.sql(f"ALTER TABLE f1_processed.results DROP IF EXISTS PARTITION (race_id = {race_id_list.race_id})")

In [0]:
# results_with_columns_df.write.mode("append").partitionBy("race_id").format("parquet").saveAsTable("f1_processed.results")
# display(spark.read.parquet(f"{processed_folder_path}/results"))

Method 2

In [0]:
# overwrite_partition(results_with_columns_df, "f1_processed", "results", "race_id")

In [0]:
merge_condition = "tgt.result_id = src.result_id AND tgt.race_id = src.race_id"
merge_delta_data(results_with_columns_df, "f1_processed", "results", processed_folder_path, merge_condition, "race_id")

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql
SELECT race_id, COUNT(*)
FROM f1_processed.results GROUP BY race_id
ORDER BY race_id DESC